# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from pathlib import Path
from newsapi import NewsApiClient

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\bmccr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Check to make sure your .env is loaded
load_dotenv()

True

In [3]:
# Read your api key environment variable
news_api_key = os.getenv("NEWS_API")
type(news_api_key)

str

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key = news_api_key)

In [9]:
# Fetch the Bitcoin news articles
btc_art = newsapi.get_everything(q = "Bitcoin", language = "en")
btc_art

{'status': 'ok',
 'totalResults': 9430,
 'articles': [{'source': {'id': 'engadget', 'name': 'Engadget'},
   'author': 'https://www.engadget.com/about/editors/saqib-shah',
   'title': 'El Salvador becomes the first country to approve Bitcoin as legal currency',
   'description': 'El Salvador has voted to adopt Bitcoin as legal tender..',
   'url': 'https://www.engadget.com/el-salvador-bitcoin-legal-tender-120440725.html',
   'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2021-06/84c12d30-c919-11eb-a6ff-cedcc0c31aae',
   'publishedAt': '2021-06-09T12:04:40Z',
   'content': "El Salvador's President Nayib Bukele has made good on his promise to adopt Bitcoin as legal tender. Officials in the Central American country's congress voted to accept the cryptocurrency by a majori… [+1414 chars]"},
  {'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Matt Novak',
   'title': 'El Salvador Becomes First Country to Recognize Bitcoin as Currency',
   'description': 'El Salvador 

In [8]:
# Fetch the Ethereum news articles
eth_art = newsapi.get_everything(q = "Ethereum", language = "en")
eth_art

{'status': 'ok',
 'totalResults': 2480,
 'articles': [{'source': {'id': 'mashable', 'name': 'Mashable'},
   'author': 'Joseph Green',
   'title': 'This blockchain development course bundle is on sale for 97% off',
   'description': 'TL;DR: The Cryptocurrency with Ethereum and Solidity Blockchain Developer Bundle is on sale for £21.25 as of June 17, saving you 97% on list price.\n\nIs everyone you know investing in cryptocurrency? The Cryptocurrency with Ethereum and Solidity Blockchain Dev…',
   'url': 'https://mashable.com/uk/shopping/june-17-cryptocurrency-ethereum-and-solidity-blockchain-developer-bundle/',
   'urlToImage': 'https://mondrian.mashable.com/2021%252F06%252F17%252Fa4%252Fd95c8d39ee014266beea8c6b9d053d19.1c765.jpg%252F1200x630.jpg?signature=13rMcdTv3FsupfDx5m8coAhsN-4=',
   'publishedAt': '2021-06-17T04:05:00Z',
   'content': 'TL;DR: The Cryptocurrency with Ethereum and Solidity Blockchain Developer Bundle is on sale for £21.25 as of June 17, saving you 97% on list price

In [13]:
# Create function that will get sentiments of articles from newsapi
# Create the Bitcoin sentiment scores DataFrame
def get_sentiments(article):
    sentiments = []
    for x in article["articles"]:
        title = x["title"]
        text = x["content"]
        date = x["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        positive = sentiment["pos"]
        neutral = sentiment["neu"]
        negative = sentiment["neg"]
        sentiments.append({
            "Title": title,
            "Text": text,
            "Date": date,
            "Compound": compound,
            "Positive": positive,
            "Neutral": neutral,
            "Negative": negative,
        })
            
    return pd.DataFrame(sentiments)

btc_sentiments = get_sentiments(btc_art)
btc_sentiments.head()

,Title,Text,Date,Compound,Positive,Neutral,Negative
0,El Salvador becomes the first country to appro...,El Salvador's President Nayib Bukele has made ...,2021-06-09T12:04:40Z,0.8402,0.282,0.718,0.000
1,El Salvador Becomes First Country to Recognize...,El Salvador has become the first country in th...,2021-06-09T10:00:00Z,0.1280,0.043,0.957,0.000
2,Elon Musk says Tesla will resume Bitcoin purch...,"It's all about clean energy, it seems. \r\nElo...",2021-06-14T07:15:49Z,0.6908,0.169,0.831,0.000
3,Kaseya ransomware attackers demand $70 million...,Filed under:\r\nThe supply chain attack has re...,2021-07-05T19:45:10Z,-0.5719,0.111,0.705,0.184
4,Bitcoin: El Salvador makes cryptocurrency lega...,image captionThe move means bitcoin will be ac...,2021-06-09T08:27:58Z,0.2732,0.060,0.940,0.000


In [14]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = get_sentiments(eth_art)
eth_sentiments.head()

,Title,Text,Date,Compound,Positive,Neutral,Negative
0,This blockchain development course bundle is o...,TL;DR: The Cryptocurrency with Ethereum and So...,2021-06-17T04:05:00Z,0.0000,0.000,1.000,0.000
1,Classic memes that have sold as NFTs,It wasn't long ago that your average person ha...,2021-06-20T19:28:07Z,-0.2960,0.000,0.939,0.061
2,Ethereum creator Vitalik Buterin made more tha...,This article was translated from our Spanish e...,2021-06-10T19:12:00Z,-0.3400,0.000,0.934,0.066
3,Ethereum now has more active addresses than bi...,Bitcoin and Ethereum\r\nYuriko Nakao\r\nEther ...,2021-07-02T15:49:56Z,0.3612,0.110,0.849,0.041
4,More than $13 billion worth of ether has been ...,"Ether holders have ""staked"" more than $13 bill...",2021-07-05T12:54:02Z,0.7717,0.194,0.806,0.000


In [15]:
# Describe the Bitcoin Sentiment
btc_sentiments.describe()

,Compound,Positive,Neutral,Negative
count,20.000000,20.000000,20.000000,20.000000
mean,0.068570,0.065050,0.895250,0.039650
std,0.387975,0.071415,0.084746,0.053571
min,-0.612400,0.000000,0.705000,0.000000
25%,-0.220025,0.000000,0.840250,0.000000
50%,0.102600,0.052500,0.922000,0.000000
75%,0.220025,0.105500,0.955500,0.064750
max,0.840200,0.282000,1.000000,0.184000


In [16]:
# Describe the Ethereum Sentiment
eth_sentiments.describe()

,Compound,Positive,Neutral,Negative
count,20.000000,20.000000,20.000000,20.000000
mean,0.063475,0.040500,0.934350,0.025150
std,0.356132,0.058708,0.053135,0.032347
min,-0.381800,0.000000,0.806000,0.000000
25%,-0.273200,0.000000,0.920000,0.000000
50%,0.000000,0.000000,0.936500,0.000000
75%,0.361200,0.074750,0.961000,0.060250
max,0.771700,0.194000,1.000000,0.080000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin (0.065050)

Q: Which coin had the highest compound score?

A: Bitcoin (0.840200)

Q. Which coin had the highest positive score?

A: Bitcoin (0.282000)

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [18]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [19]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
stopwords = set(stopwords.words("english"))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [56]:
# Tokenizes Text from DataFrame
def tokenizer(text):
    regex = re.compile("[^a-zA-Z0-9]")
    text['Tokens'] = ""

    for i in range(len(text)):
        
        # Remove the punctuation from text
        clean = regex.sub(" ", text['Text'].loc[i])
        
        # Tokenizes text
        words = word_tokenize(clean)
        
        # Convert the words to lowercase & Remove the stop words
        words_lower = [x.lower() for x in words if x.lower() not in stopwords]
        # Lemmatize words into root words
        words_lemmatized = [lemmatizer.lemmatize(x) for x in words_lower]
        text['Tokens'][i] = words_lemmatized
    
    return text

In [57]:
# Create a new tokens column for Bitcoin
btc_art_tokenized = tokenizer(btc_sentiments)
btc_art_tokenized.head()

<ipython-input-56-dba075d6f9f3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Tokens'][i] = words_lemmatized


,Title,Text,Date,Compound,Positive,Neutral,Negative,Tokens
0,El Salvador becomes the first country to appro...,El Salvador's President Nayib Bukele has made ...,2021-06-09T12:04:40Z,0.8402,0.282,0.718,0.000,"[el, salvador, president, nayib, bukele, made,..."
1,El Salvador Becomes First Country to Recognize...,El Salvador has become the first country in th...,2021-06-09T10:00:00Z,0.1280,0.043,0.957,0.000,"[el, salvador, become, first, country, world, ..."
2,Elon Musk says Tesla will resume Bitcoin purch...,"It's all about clean energy, it seems. \r\nElo...",2021-06-14T07:15:49Z,0.6908,0.169,0.831,0.000,"[clean, energy, seems, elon, musk, tesla, caus..."
3,Kaseya ransomware attackers demand $70 million...,Filed under:\r\nThe supply chain attack has re...,2021-07-05T19:45:10Z,-0.5719,0.111,0.705,0.184,"[filed, supply, chain, attack, reached, thousa..."
4,Bitcoin: El Salvador makes cryptocurrency lega...,image captionThe move means bitcoin will be ac...,2021-06-09T08:27:58Z,0.2732,0.060,0.940,0.000,"[image, captionthe, move, mean, bitcoin, accep..."


In [58]:
# Create a new tokens column for Ethereum
eth_art_tokenized = tokenizer(eth_sentiments)
eth_art_tokenized.head()

<ipython-input-56-dba075d6f9f3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Tokens'][i] = words_lemmatized


,Title,Text,Date,Compound,Positive,Neutral,Negative,Tokens
0,This blockchain development course bundle is o...,TL;DR: The Cryptocurrency with Ethereum and So...,2021-06-17T04:05:00Z,0.0000,0.000,1.000,0.000,"[tl, dr, cryptocurrency, ethereum, solidity, b..."
1,Classic memes that have sold as NFTs,It wasn't long ago that your average person ha...,2021-06-20T19:28:07Z,-0.2960,0.000,0.939,0.061,"[long, ago, average, person, clue, nft, nowada..."
2,Ethereum creator Vitalik Buterin made more tha...,This article was translated from our Spanish e...,2021-06-10T19:12:00Z,-0.3400,0.000,0.934,0.066,"[article, translated, spanish, edition, using,..."
3,Ethereum now has more active addresses than bi...,Bitcoin and Ethereum\r\nYuriko Nakao\r\nEther ...,2021-07-02T15:49:56Z,0.3612,0.110,0.849,0.041,"[bitcoin, ethereum, yuriko, nakao, ether, over..."
4,More than $13 billion worth of ether has been ...,"Ether holders have ""staked"" more than $13 bill...",2021-07-05T12:54:02Z,0.7717,0.194,0.806,0.000,"[ether, holder, staked, 13, billion, worth, cr..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---